In [12]:
import numpy as np
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score, roc_auc_score

from sklearn.model_selection import cross_val_score, GridSearchCV

import pickle


In [13]:
#a

In [14]:
class uji(): 
    def __init__(self,data): 
        df0 = pd.read_csv('../data2_smpl30_2/uji%snorm.csv'%data, delimiter= ",", header=None)
        df0=df0.fillna(0)
        self.com=df0[0]
        self.prot=df0[1282]
        self.x =df0.drop([0,1282], axis=1)
        df0[1300]=0
        
        self.y=df0[1300]
        self.dt=data

def dtuji(data): 
    return uji(data) 


dtf =dtuji('mbafika') #dtf.com, dtf.prot, dtf.x, dtf.y, dtf.dt
dtl =dtuji('mbalinda') #dtl.com, dtl.prot, dtl.x, dtl.y dtl.dt


In [15]:
dtg =dtuji('gabung') #dtl.com, dtl.prot, dtl.x, dtl.y dtl.dt

In [16]:

class latih(): 
    def __init__(self,data): 
        df = pd.read_csv("../data2_smpl30_2/%s.csv"%data, delimiter=',')
        df.head()
        self.X = df.drop(['0','1282','0.1'], axis=1)
        self.Y = df['0.1']
        self.dt=data

def dtlatih(data): 
    return latih(data) 
df1= dtlatih('latih_sampled30_1') #df1.X,df1.Y,df1.dt
df2= dtlatih('latih_sampled30_2') #df2.X,df2.Y,df2.dt
df3= dtlatih('latih_sampled30_3') #df3.X,df3.Y,df3.dt
df4= dtlatih('latih_sampled30_4') #df4.X,df4.Y,df4.dt
df5= dtlatih('latih_sampled30_5') #df5.X,df5.Y,df5.dt

/home/biofarmaka/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
du5=dtlatih('uji_sampled30_5') #df5.X,df5.Y,df5.dt
du4=dtlatih('uji_sampled30_4') #df5.X,df5.Y,df5.dt
du3=dtlatih('uji_sampled30_3') #df5.X,df5.Y,df5.dt
du2=dtlatih('uji_sampled30_2') #df5.X,df5.Y,df5.dt
du1=dtlatih('uji_sampled30_1') #df5.X,df5.Y,df5.dt

In [18]:
scoring = {'AUC': 'roc_auc', 'Accuracy': 'accuracy','F-measure':'f1','Precision':'precision', 'Recall':'recall'}


In [19]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier

In [20]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(640, input_dim=1281, init='normal', activation='relu'))
    model.add(Dense(320, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [21]:
def mlp(df):
    mlp = MLPClassifier(max_iter=1000,hidden_layer_sizes=(640,320,),activation= 'relu',
                        solver= 'adam')
#    model = Sequential()
#    model.add(Dense(640, input_dim=1281, init='uniform', activation='relu'))
#    model.add(Dense(320, init='uniform', activation='relu'))
#    model.add(Dense(1, init='uniform', activation='sigmoid'))
#    estimators = []
#    estimators.append(('standardize', StandardScaler()))
#    estimators.append(('mlp', KerasClassifier(build_fn=create_model, nb_epoch=100, batch_size=128, verbose=0,class_weight="balanced")))
#    pipeline = Pipeline(estimators)
#    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
#    results = cross_val_score(pipeline, df.X, df.Y, cv=kfold)
#    print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    parameter_space = {
        #'hidden_layer_sizes': [(640,320,)],
        #'activation': ['relu'],
        #'solver': ['adam'],
        'alpha': [0.5,0.01],
        'learning_rate': ['constant','adaptive'],
        'batch_size':[32,64,128,256,512]
    }
    mlp = GridSearchCV(mlp, parameter_space, n_jobs=15, cv=5,scoring=scoring, refit='AUC',return_train_score=True)
    mlp.fit(df.X, df.Y)
    print('done_training')
    filename = '../data2_smpl30_2/model/mlp_{0}'.format(df.dt)
    pickle.dump(mlp, open(filename, 'wb'))
    print('done_savemodel')
    return mlp

In [22]:
#rf5_100n_1000m.cv_results_#['mean_test_score']
#stds = clf.cv_results_['std_test_score']
def print_cv_results_mlp(model,o):
    rf = pickle.load(open('../data2_smpl30_2/model/{0}_{1}'.format(model.__name__,o.dt), 'rb'))
    print('{0}_{1}'.format(model.__name__,o.dt))
    print('Best parameters found:', rf.best_params_)
    print('AUC :',rf.cv_results_['mean_test_AUC'])
    print('Accuracy :',rf.cv_results_['mean_test_Accuracy'])
    print('F-measure :',rf.cv_results_['mean_test_F-measure'])
    print('Precision :',rf.cv_results_['mean_test_Precision'])
    print('Recall :',rf.cv_results_['mean_test_Recall'])

In [23]:
def report_rf(model,df,d):
    rf = pickle.load(open('../data2_smpl30_2/model/{0}_latih_sampled30_{1}'.format(model.__name__,d), 'rb'))
    pred_x = rf.predict(df.X)
    
    print('model file : {0}_{1}'.format(model.__name__,d))
    #print('Parameters used:', rf.get_params() )
    print('Best parameters found:', rf.best_params_)
    print('CV RESULT :')
    print('AUC :',rf.cv_results_['mean_test_AUC'][(rf.cv_results_['rank_test_AUC']-1)[0]])
    print('Accuracy :',rf.cv_results_['mean_test_Accuracy'][(rf.cv_results_['rank_test_Accuracy']-1)[0]])
    print('F-measure :',rf.cv_results_['mean_test_F-measure'][(rf.cv_results_['rank_test_F-measure']-1)[0]])
    print('Precision :',rf.cv_results_['mean_test_Precision'][(rf.cv_results_['rank_test_Precision']-1)[0]])
    print('Recall :',rf.cv_results_['mean_test_Recall'][(rf.cv_results_['rank_test_Recall']-1)[0]])
    #print('Accuracy:',accuracy_score(df.Y, pred_x))
    #print('f1:',f1_score(df.Y, pred_x))
    #print('precision:',precision_score(df.Y, pred_x))
    #print('recall:',recall_score(df.Y, pred_x))
    #print('roc_auc:',roc_auc_score(df.Y, pred_x))
    #print('Confusion matrix:')
    #print(confusion_matrix(df.Y, pred_x))
    #print('Results on the train set:')
    #print(classification_report(df.Y, pred_x))
    return rf
    #pred_n = rf.predict(cX)
    #print('Results on the all neg data:')
    #print(classification_report(cY, pred_n))
    
def prediksi(md,model,p,data,d):
    pred_x = pd.DataFrame(md.predict_proba(data.x))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold {0} ".format(p))
    print("model_hasilsmote_{0} ".format(d))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(confusion_matrix(data.y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    pred[2] = data.com
    pred[3] = data.prot
    #print(pred.head())
    #print(prob_x)
    pred[4] = pred_x[1]
    #print(pred.head())
    #print(pd.DataFrame(prob_x)[1].head())
    pred.to_excel('../data2_smpl30_2/hasil_pred/{0}_{1}p_{2}_latih_sampled30_{3}.xlsx'.format(model.__name__,p,data.dt,d))
    
def prediksi2(md,model,p,data,d):
    pred_x = pd.DataFrame(md.predict_proba(data.X))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold {0} ".format(p))
    print("model_hasilsmote_{0} ".format(d))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(data.Y.value_counts())
    print(confusion_matrix(data.Y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    #pred[2] = data.com
    #pred[3] = data.prot
    print('Accuracy:',accuracy_score(data.Y, prob_x[1]))
    print('f1:',f1_score(data.Y, prob_x[1]))
    print('precision:',precision_score(data.Y, prob_x[1]))
    print('recall:',recall_score(data.Y, prob_x[1]))
    print('roc_auc:',roc_auc_score(data.Y, prob_x[1]))
    print('Confusion matrix:')
    print(confusion_matrix(data.Y, prob_x[1]))
    print('Results on the train set:')
    print(classification_report(data.Y, prob_x[1]))
    #print(pred.head())
    #print(prob_x)
    #pred[4] = pred_x[1]
    #print(pred.head())
    #print(pd.DataFrame(prob_x)[1].head())
    #pred.to_excel('../data2_smpl30_2/hasil_pred/{0}_{1}p_{2}_latih_sampled30_{3}.xlsx'.format(model.__name__,p,data.dt,d))

In [24]:
def fitn3pred(model,p,datalatih,datauji1,datauji2,datauji3,d):
    mlp(datalatih)
    md=report_rf(model,datalatih,d)
    prediksi(md,model,p,datauji1,d)
    prediksi(md,model,p,datauji2,d)
    prediksi2(md,model,p,datauji3,d)
def fitn2pred(model,p,datalatih,datauji1,datauji2,d):
    mlp(datalatih)
    md=report_rf(model,datalatih,d)
    prediksi(md,model,p,datauji1,d)
    prediksi(md,model,p,datauji2,d)
def fitnpred(model,p,datalatih,datauji1,d):
    mlp(datalatih)
    md=report_rf(model,datalatih,d)
    prediksi(md,model,p,datauji1,d)

In [25]:
fitn3pred(mlp,0.5,df1,dtf,dtl,du1,1)


done_training
done_savemodel
model file : mlp_1
Best parameters found: {'alpha': 0.5, 'batch_size': 256, 'learning_rate': 'adaptive'}
CV RESULT :
AUC : 0.9945650736500837
Accuracy : 0.9791597486805308
F-measure : 0.9782794593899654
Precision : 0.9707172216688003
Recall : 0.9960651289009498
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbafika  : (0,1)
[[138 102]
 [  0   0]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  mbalinda  : (0,1)
[[1753 1295]
 [   0    0]]
threshold 0.5 
model_hasilsmote_1 
hasil prediksi  uji_sampled30_1  : (0,1)
0    3537
1    3086
Name: 0.1, dtype: int64
[[3423  114]
 [   0 3086]]
Accuracy: 0.9827872565302733
f1: 0.9818644607063315
precision: 0.964375
recall: 1.0
roc_auc: 0.9838846480067854
Confusion matrix:
[[3423  114]
 [   0 3086]]
Results on the train set:
              precision    recall  f1-score   support

           0       1.00      0.97      0.98      3537
           1       0.96      1.00      0.98      3086

    accuracy                

In [26]:
def mlp(df):
    mlp = MLPClassifier(max_iter=1000, batch_size=128,hidden_layer_sizes=(640,320,),activation= 'relu',
                        solver= 'adam')
#    model = Sequential()
#    model.add(Dense(640, input_dim=1281, init='uniform', activation='relu'))
#    model.add(Dense(320, init='uniform', activation='relu'))
#    model.add(Dense(1, init='uniform', activation='sigmoid'))
#    estimators = []
#    estimators.append(('standardize', StandardScaler()))
#    estimators.append(('mlp', KerasClassifier(build_fn=create_model, nb_epoch=100, batch_size=128, verbose=0,class_weight="balanced")))
#    pipeline = Pipeline(estimators)
#    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
#    results = cross_val_score(pipeline, df.X, df.Y, cv=kfold)
#    print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    parameter_space = {
        #'hidden_layer_sizes': [(640,320,)],
        #'activation': ['relu'],
        #'solver': ['adam'],
        'alpha': [0.5],
        'learning_rate': ['adaptive'],
        'batch_size':[256]
    }
    mlp = GridSearchCV(mlp, parameter_space, n_jobs=3, cv=5,scoring=scoring, refit='AUC',return_train_score=True)
    mlp.fit(df.X, df.Y)
    print('done_training')
    filename = '../data2_smpl30_2/model/mlp_{0}'.format(df.dt)
    pickle.dump(mlp, open(filename, 'wb'))
    print('done_savemodel')
    return mlp

In [27]:
fitn3pred(mlp,0.5,df2,dtf,dtl,du2,2)
fitn3pred(mlp,0.5,df3,dtf,dtl,du3,3)
fitn3pred(mlp,0.5,df4,dtf,dtl,du4,4)
fitn3pred(mlp,0.5,df5,dtf,dtl,du5,5)

done_training
done_savemodel
model file : mlp_2
Best parameters found: {'alpha': 0.5, 'batch_size': 256, 'learning_rate': 'adaptive'}
CV RESULT :
AUC : 0.995868440564764
Accuracy : 0.9824603845175315
F-measure : 0.9817814959655253
Precision : 0.9645734772207024
Recall : 0.9997254632807138
threshold 0.5 
model_hasilsmote_2 
hasil prediksi  mbafika  : (0,1)
[[163  77]
 [  0   0]]
threshold 0.5 
model_hasilsmote_2 
hasil prediksi  mbalinda  : (0,1)
[[1965 1083]
 [   0    0]]
threshold 0.5 
model_hasilsmote_2 
hasil prediksi  uji_sampled30_2  : (0,1)
0    3450
1    3173
Name: 0.1, dtype: int64
[[3325  125]
 [   0 3173]]
Accuracy: 0.9811263777744225
f1: 0.9806830474424354
precision: 0.962098241358399
recall: 1.0
roc_auc: 0.9818840579710144
Confusion matrix:
[[3325  125]
 [   0 3173]]
Results on the train set:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98      3450
           1       0.96      1.00      0.98      3173

    accuracy        